In [ ]:
import sys
sys.path.append('/Users/ik/Desktop/zepeto')
from a_python.rigging_class.rig_hier_maya_torch import *
import json
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pickle as pkl
import scipy
import torch
import math
import time
import pytorch3d
from a_python.utils.rot import _rot_base
from pytorch3d.io import load_objs_as_meshes, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    OpenGLOrthographicCameras,
    FoVPerspectiveCameras, 
    PointLights, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    SoftSilhouetteShader,
    SoftPhongShader
)
from pytorch3d.renderer.mesh import rasterize_meshes
from pytorch3d.renderer.mesh.rasterize_meshes import rasterize_meshes_python

from pytorch3d.renderer.blending import BlendParams
from pytorch3d.renderer.blending import sigmoid_alpha_blend, _sigmoid_alpha
import copy

%matplotlib widget
body = rig_class(65)

_vertices = copy.deepcopy(vertices)

#### DR optimize function

In [11]:
def upperbody_rigging_for_joint_matching_only_body(body,_vertices, opt_upper_scale, opt_upper_rot):
    '''상체 joint_matching용 rigging'''
    body_init = copy.deepcopy(body)
    _vertices_opt = copy.deepcopy(_vertices)

    # spine
    body_init.childs[0].scale_iso_hier(opt_upper_scale[0], _vertices_opt)
    body_init.childs[0].rot(_rot_base('z', opt_upper_rot[0]), _vertices_opt, inplace=False)
    # chestupper
    body_init.childs[0].childs[0].childs[0].rot(_rot_base('z', opt_upper_rot[1]), _vertices_opt, inplace=False)
    # shoulder
    body_init.childs[0].childs[0].childs[0].childs[1].scale_iso_hier(opt_upper_scale[1], _vertices_opt)
    body_init.childs[0].childs[0].childs[0].childs[2].scale_iso_hier(opt_upper_scale[1], _vertices_opt)

    # joint
    joint = []
    # Left
    ## upperarm
    joint.append(body_init.childs[0].childs[0].childs[0].childs[1].childs[0].T_mat[:3,3][None,:])
    # Right
    ## upperarm
    joint.append(body_init.childs[0].childs[0].childs[0].childs[2].childs[0].T_mat[:3,3][None,:])
    
    joint = torch.cat(joint, dim=0)
    joint_aligned = joint * torch.tensor([1,-1,1]) - _vertices_opt[3106,:]* torch.tensor([1,-1,1]) + torch.tensor([crotch[0], crotch[1], 0])
    
    return body_init, _vertices_opt, joint_aligned

def loss_dr_upper_only_body(body, _vertices, opt_upper_scale, opt_upper_rot, gt_joint):
    body_init, _vertices_opt, joint = upperbody_rigging_for_joint_matching_only_body(body, _vertices, opt_upper_scale, opt_upper_rot)
    loss = torch.abs(gt_joint[:,:2] - joint[:,:2]).sum()
    return loss

In [32]:
body = rig_class(65)
body_time = copy.deepcopy(body)
_vertices_time = copy.deepcopy(vertices)


def time_check(body_init, _vertices_opt):
    # spine, shoulder
    opt_upper_scale = torch.nn.Parameter(torch.tensor([1., 1.]))
    # spine_z, chestuppper_z
    opt_upper_rot = torch.nn.Parameter(torch.tensor([1., 1.])) 
    # spine, chest, chestupper scale
    opt_x = torch.nn.Parameter(torch.tensor([1., 1., 1.])) 
    # optimize paramters
    body_init.childs[0].scale_y(opt_x[0], _vertices_opt)
    body_init.childs[0].childs[0].scale_y(opt_x[1], _vertices_opt)
    body_init.childs[0].childs[0].childs[0].scale_y(opt_x[2], _vertices_opt)


    # Prior result
    ## spine
    t1 = time.time()
    body_init.childs[0].scale_iso_hier(opt_upper_scale[0], _vertices_opt)
    t2 = time.time()
    body_init.childs[1].scale_iso_hier(opt_upper_scale[0], _vertices_opt)
    # body_init.childs[0].rot(_rot_base('z', opt_upper_rot[0]), _vertices_opt, inplace=False)
    ## chestupper
    t3 = time.time()
    body_init.childs[0].childs[0].childs[0].rot(_rot_base('z', opt_upper_rot[1]), _vertices_opt, inplace=False)
    t4 = time.time()
    ## shoulder
    body_init.childs[0].childs[0].childs[0].childs[1].scale_iso_hier(opt_upper_scale[1], _vertices_opt)
    body_init.childs[0].childs[0].childs[0].childs[2].scale_iso_hier(opt_upper_scale[1], _vertices_opt)

    # print("time 1:", t2-t1)
    # print("time 2:", t3-t2)
    # print("time 3:", t4-t3)


def time_check_no_grad(body_init, _vertices_opt):
    with torch.no_grad():
        # spine, shoulder
        opt_upper_scale = torch.nn.Parameter(torch.tensor([1., 1.]))
        # spine_z, chestuppper_z
        opt_upper_rot = torch.nn.Parameter(torch.tensor([1., 1.])) 
        # spine, chest, chestupper scale
        opt_x = torch.nn.Parameter(torch.tensor([1., 1., 1.])) 
        # optimize paramters
        body_init.childs[0].scale_y(opt_x[0], _vertices_opt)
        body_init.childs[0].childs[0].scale_y(opt_x[1], _vertices_opt)
        body_init.childs[0].childs[0].childs[0].scale_y(opt_x[2], _vertices_opt)
        

        # Prior result
        ## spine
        t1 = time.time()
        body_init.childs[0].scale_iso_hier(opt_upper_scale[0], _vertices_opt)
        t2 = time.time()
        body_init.childs[1].scale_iso_hier(opt_upper_scale[0], _vertices_opt)
        # body_init.childs[0].rot(_rot_base('z', opt_upper_rot[0]), _vertices_opt, inplace=False)
        ## chestupper
        t3 = time.time()
        body_init.childs[0].childs[0].childs[0].rot(_rot_base('z', opt_upper_rot[1]), _vertices_opt, inplace=False)
        t4 = time.time()
        ## shoulder
        body_init.childs[0].childs[0].childs[0].childs[1].scale_iso_hier(opt_upper_scale[1], _vertices_opt)
        body_init.childs[0].childs[0].childs[0].childs[2].scale_iso_hier(opt_upper_scale[1], _vertices_opt)
    print("time total:", t4-t1)

body_time = copy.deepcopy(body)
_vertices_time = copy.deepcopy(vertices)
time_check(body_time, _vertices_time)

tmp = body_time.childs[0].childs[0].childs[0].childs[2]
print(tmp.T_mat )
print(tmp.head)
print(tmp.dir_longi)
print(tmp.dir_y)
print(tmp.dir_z )


tensor([[ 6.8997e-01,  7.2002e-01,  7.4207e-02, -7.1548e+00],
        [ 7.1665e-01, -6.9393e-01,  6.9782e-02,  7.3362e+01],
        [ 1.0174e-01,  5.0328e-03, -9.9480e-01, -1.3653e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       grad_fn=<MmBackward0>)
tensor([-7.1548, 73.3624, -1.3653], grad_fn=<SelectBackward0>)
tensor([0.6900, 0.7166, 0.1017], grad_fn=<DivBackward0>)
tensor([ 0.7200, -0.6939,  0.0050], grad_fn=<DivBackward0>)
tensor([ 0.0742,  0.0698, -0.9948], grad_fn=<DivBackward0>)


#### 시간 늘어나기 전

In [7]:
body_time = copy.deepcopy(body)
_vertices_time = copy.deepcopy(vertices)
time_check(body_time, _vertices_time)

time total: 0.009732246398925781


#### 시간 늘어난 후

# 어디가 오래걸리나

In [9]:
def img_upperbody_maya_joints(keypts):
    '''maya관절 길이비율대로 상체 조인트 추가 : chest upper, chest, spine'''
    img_hip = (keypts[9,:2] + keypts[12,:2])/2
    img_neck = (keypts[2,:2] + keypts[5,:2])/2
    img_spine = (img_neck - img_hip) * len_joint_upper[-1] + img_hip
    img_chest = (img_neck - img_hip) * (len_joint_upper[-1] + len_joint_upper[-2]) + img_hip
    img_chsetupper = (img_neck - img_hip) * (len_joint_upper[-1] + len_joint_upper[-2] + len_joint_upper[-3])  + img_hip
    return img_hip, img_neck, img_spine, img_chest, img_chsetupper

def fitting_based_on_upperbody_joints(body, img_spine, img_hip, img_neck, _vertices):
    '''img_upperbody_maya_joints 함수 결과 사용해서 메쉬 스케일링으로 상체 조인트위치 맞춤'''
    # hip to spine 거리
    _len_spine_maya = np.linalg.norm(body.head - body.childs[0].head) 
    _len_spine_img = np.linalg.norm(img_spine - img_hip)

    # 상체 조인트 맞춤
    _spine_trans = -(_len_spine_maya - _len_spine_img) * \
        ((-body.head + body.childs[0].head) / np.linalg.norm(body.head - body.childs[0].head))
    body.childs[0].trans_head(_spine_trans, _vertices)
    _scale_spine_hier = (np.linalg.norm(img_hip - img_neck) * len_joint_upper[-1]) / \
        (np.linalg.norm(body.childs[0].head - body.childs[0].childs[0].head))
    body.childs[0].scale_iso_hier(_scale_spine_hier, _vertices)

def _rot_base(axis, angle):
    zero = torch.zeros_like(angle)
    one = torch.ones_like(angle)
    if axis == 'x':
        return torch.stack(
            [
                torch.stack([one, zero, zero]),
                torch.stack([zero, torch.cos(angle), -torch.sin(angle)]),
                torch.stack([zero, torch.sin(angle), torch.cos(angle)])
            ]
        )

    if axis == 'y':
        return torch.stack(
            [
                torch.stack([torch.cos(angle), zero, torch.sin(angle)]),
                torch.stack([zero, one, zero]),
                torch.stack([-torch.sin(angle), zero, torch.cos(angle)])
            ]
        )

    if axis == 'z':
        return torch.stack(
            [
                torch.stack([torch.cos(angle), -torch.sin(angle), zero]),
                torch.stack([torch.sin(angle), torch.cos(angle), zero]),
                torch.stack([zero, zero, one]),
            ]
        )

def deform_mesh(opt_x, body, _vertices_opt):
    body.childs[1].childs[0].scale_y(opt_x[2], _vertices_opt)
    body.childs[1].childs[0].scale_z(opt_x[2], _vertices_opt)
    body.childs[1].childs[0].childs[0].scale_y(opt_x[3], _vertices_opt)
    body.childs[1].childs[0].childs[0].scale_z(opt_x[3], _vertices_opt)
    
    body.childs[1].childs[1].scale_y(opt_x[2], _vertices_opt)
    body.childs[1].childs[1].scale_z(opt_x[2], _vertices_opt)
    body.childs[1].childs[1].childs[0].scale_y(opt_x[3], _vertices_opt)
    body.childs[1].childs[1].childs[0].scale_z(opt_x[3], _vertices_opt)

    body.childs[1].childs[0].rot(_rot_base('z', opt_x[0]), _vertices_opt, inplace=False)
    body.childs[1].childs[1].rot(_rot_base('z', opt_x[1]), _vertices_opt, inplace=False)

def render_changed_mesh_subimg(body, mesh, _vertices, opt_x, img_size, rasterize_size, crotch_subimg):
    # mesh change
    _vertices_opt = copy.deepcopy(_vertices)
    body_copy = copy.deepcopy(body)
    deform_mesh(opt_x, body_copy, _vertices_opt)

    move = torch.tensor([rasterize_size/2, rasterize_size/2, 0]) - crotch_subimg

    # 허벅지 crop에 맞게 cropped rasterize
    mesh._verts_packed = (
        _vertices_opt - _vertices_opt[3106,:]\
        + torch.tensor([0, 0, img_size/2]) + move
        ) / (rasterize_size / 2)
    
    fragments = rasterize_meshes(mesh, image_size = rasterize_size, blur_radius=0, faces_per_pixel=8, perspective_correct= False, )
    blend_params = BlendParams()
    res = _sigmoid_alpha(fragments[3], fragments[0], blend_params.sigma)
    return res

def loss_dr(img_target, opt_x, body, _vertices, crotch, crotch_subimg, img_size,rasterize_size, mesh, img_weight = None):
    res = render_changed_mesh_subimg(body, mesh, _vertices, opt_x, img_size, rasterize_size, crotch_subimg)

    # 넘치는거 막기 위한 가중치 있을 떄
    if img_weight is None:
        loss = (torch.abs(img_target - res)).sum()
    else:
        loss = (img_weight * torch.abs(img_target - res)).sum()
    
    return loss

# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")


def visualize_vertices_torch(_vertices_opt, body_init, __color=np.array([0.5,0.3,1]), _chosen_idx=None):
    _list_joint = []
    pcd = o3d.geometry.PointCloud()
    _vertices_draw = np.asarray(copy.deepcopy(_vertices_opt.detach()))

    pcd.points = o3d.utility.Vector3dVector(_vertices_draw)
    _color = np.ones([9067,3])*__color
    
    if _chosen_idx is not None:
        _color[_chosen_idx,:] = np.array([1, 0, 0])
    pcd.colors = o3d.utility.Vector3dVector(_color)
    
    _list_joint.append(pcd)
    body_init.draw_joint(_list_joint)
    o3d.visualization.draw_geometries(_list_joint)


# def visualize_vertices_torch_from_param(_vertices, body_init, opt_upper_scale, opt_upper_rot, __color=np.array([0.5,0.3,1])):
#     _list_joint = []
#     pcd = o3d.geometry.PointCloud()
#     body_init, _vertices_opt,_ = upperbody_rigging_for_joint_matching(body,_vertices, opt_upper_scale, opt_upper_rot)
#     _vertices_draw = np.asarray(copy.deepcopy(_vertices_opt.detach()))

#     pcd.points = o3d.utility.Vector3dVector(_vertices_draw)
#     _color = np.ones([9067,3])*__color
#     pcd.colors = o3d.utility.Vector3dVector(_color)

#     _list_joint.append(pcd)
#     body_init.draw_joint(_list_joint)
#     o3d.visualization.draw_geometries(_list_joint)




In [10]:
# 사진 index

idx_img = 11
idx_img = 8

# optimize parameters
n_iter = 50
alpha_loss_excede_mask = 10          # 마스크 넘어가는거에 얼마나 가중치 줄건지

visualize_pcd = False

%matplotlib widget

time_start = time.time()
folder = idx_img
path_img = "/Users/ik/Downloads/test_set_new/{}/test_resized.png".format(folder)
path_json = "/Users/ik/Downloads/test_set_new/{}/test_resized_keypoints.json".format(folder)
path_dp = "/Users/ik/Downloads/test_set_new/{}/dp_dump.pkl".format(folder)

with open(path_json, 'r') as f:
    data = json.load(f)
with open(path_dp, 'rb') as f:
    [img_seg, img_v, img_u,_] = pkl.load(f)
h,w = img_seg.shape
keypts = torch.tensor(data['people'][0]['pose_keypoints_2d']).reshape([-1,3])
img = cv.imread(path_img)



body = rig_class(65)


x = np.linspace(0, w-1, w)
y = np.linspace(0, h-1, h)
xx, yy = np.meshgrid(x,y, indexing='xy')


_vertices = copy.deepcopy(vertices)
# 가랑이 찾고, 하체 다리길이 맞추기용 키포인트 길이 구하기 : 가랑이부분 허벅지 두께, 무릎, 발목 폭
# image랑 _vertices의 스케일이 같아짐 
crotch = body.scaling_joint_len(_vertices, keypts, img_seg)

# 길이비율로 상체 오픈포즈에 마야 조인트 추가, 메쉬 맞추기
img_hip, img_neck, img_spine, img_chest, img_chsetupper = img_upperbody_maya_joints(keypts)
fitting_based_on_upperbody_joints(body, img_spine, img_hip, img_neck, _vertices)

# base mesh for face index
mesh = load_objs_as_meshes(['/Users/ik/Desktop/zepeto/blender/wip_find_weight.obj'], device)

img_size = h




# body class 카피 - optimization에서 사용
body_init = copy.deepcopy(body)

img_mask = (img_seg!=0).astype(int)

# 허벅지 부분 이미지
thr_surplus = 3     # bouning box 여유분
mask_leg = (img_seg == 10) + (img_seg == 9)
bbox = np.array([[xx[mask_leg].min()-thr_surplus , yy[mask_leg].min()-thr_surplus],[xx[mask_leg].max()+thr_surplus , yy[mask_leg].max()+thr_surplus]]).astype(int)
_ul, _lr = torch.max(torch.abs(torch.from_numpy(bbox) - crotch), dim=1).values

rasterize_size = _ul+_lr

bbox = np.array([[crotch[0] - _ul, crotch[1] - _ul],[crotch[0] + _lr, crotch[1] + _lr]]).astype(int)
crotch_subimg = torch.tensor([_ul, _ul, 0])


# initial rasterization and subimage aligning
_vertices_opt = copy.deepcopy(_vertices)
## 허벅지 crop에 맞게 cropped rasterize
mesh_crotch = (
    _vertices_opt[3106,:] - _vertices_opt[3106,:] \
    - torch.tensor([crotch[0], crotch[1], 0]) \
    + torch.tensor([img_size/2, img_size/2, img_size/2])
    )/img_size * 2
mesh_crotch = mesh_crotch.detach()

# _mesh_scale = img_size / rasterize_size
# _move_x = 2*_ul/(_ul + _lr) - 1 - _mesh_scale * mesh_crotch[0] 
# _move_y = 2*_ul/(_ul + _lr) - 1 - _mesh_scale * mesh_crotch[1]

move = torch.tensor([rasterize_size/2, rasterize_size/2, 0]) - crotch_subimg




##########################################################################################
##########################################################################################
##########################################################################################

# rasterization의 경우 normalize 해줘야함 [-1,1]
mesh._verts_list[0] = (
    _vertices - _vertices[3106,:] + torch.tensor([crotch[0], crotch[1], 0]) \
    - torch.tensor([img_size/2, img_size/2, -img_size/2])
    )/img_size * 2

tmp_scale = torch.tensor([1.0])
tmp_scale.requires_grad = True
mesh._verts_list[0] *= tmp_scale

##########################################################################################
##########################################################################################
##########################################################################################




fragments = rasterize_meshes(mesh, image_size = (128, 64), blur_radius=0, faces_per_pixel=8, perspective_correct= False, )
# fragments = rasterize_meshes(mesh, image_size = int(_ul + _lr), blur_radius=0, faces_per_pixel=8, perspective_correct= False, )


body_time = copy.deepcopy(body)
_vertices_time = copy.deepcopy(vertices)
time_check(body_time, _vertices_time)

scales: tensor(2.5097) , tensor(0.8982) , tensor(0.9761) , tensor(0.8403)


/opt/homebrew/lib/python3.9/site-packages/pytorch3d/io/obj_io.py:542: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


time total: 0.008342981338500977


In [35]:
#######################################################
#######################################################
body_time = copy.deepcopy(body)
_vertices_time = copy.deepcopy(vertices)
time_check(body_time, _vertices_time)
#######################################################
#######################################################


n_iter = 5

# spine, shoulder
opt_upper_scale = torch.nn.Parameter(torch.tensor([1., 1.]))
# spine_z, chestuppper_z
opt_upper_rot = torch.nn.Parameter(torch.tensor([0., 0.])) 
body_init, _vertices_opt, joint = upperbody_rigging_for_joint_matching_only_body(body, _vertices, opt_upper_scale, opt_upper_rot)

if False:
    joint_draw_aligned = np.asarray(joint.detach())

    %matplotlib widget
    plt.figure('from rigged')
    plt.imshow(img_seg)
    plt.scatter(joint_draw_aligned[:,0], joint_draw_aligned[:,1])

    plt.figure('aligned')
    plt.imshow(img_seg)
    plt.scatter(joint_draw_aligned[:,0], joint_draw_aligned[:,1])
    plt.scatter(keypts[1:8,0], keypts[1:8,1])

gt_index = [5,2]
gt_joint = keypts[gt_index,:]

#######################################################
body_time = copy.deepcopy(body)
_vertices_time = copy.deepcopy(vertices)
time_check(body_time, _vertices_time)
#######################################################

torch.autograd.set_detect_anomaly(True)

optimizer = torch.optim.SGD([{'params' : opt_upper_scale}, {'params' : opt_upper_rot}], lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam([{'params' : opt_upper_scale}, {'params' : opt_upper_rot}], lr=0.01)

opt_time = time.time()
for i in range(1):
    optimizer.zero_grad()
    loss = loss_dr_upper_only_body(body, _vertices, opt_upper_scale, opt_upper_rot, gt_joint)
    if i+1 % 10 == 0:
        print('loss:', loss)
    loss.backward()
    optimizer.step()


print(time.time() - opt_time)

#######################################################
body_time = copy.deepcopy(body)
_vertices_time = copy.deepcopy(vertices)
time_check(body_time, _vertices_time)
#######################################################


time 0: 3.0040740966796875e-05
time 1: 0.9655778408050537
time 2: 2.1457672119140625e-06
time 3: 0.004171133041381836
time 4: 0.00015878677368164062
0.9702088832855225


In [27]:
body_time = copy.deepcopy(body)
_vertices_time = copy.deepcopy(vertices)
time_check_no_grad(body_time, _vertices_time)
time_check(body_time, _vertices_time)

time total: 0.006828784942626953
time 1: 0.20571398735046387
time 2: 0.06356096267700195
time 3: 0.2627401351928711


In [29]:
print(body_init.childs[0].name)
print(body_init.childs[1].name)

spine_scale
pelvis_scale
